<a href="https://colab.research.google.com/github/tsatie/SatieGitHubTest/blob/master/0504JordanMatrixTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jordan標準形を利用した行列の累乗を求める過程の確認
SymPy と Latexify を準備する。

In [1]:
using Pkg
Pkg.add("SymPy")
Pkg.add("Latexify")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed TermInterface ─ v2.0.0
   Installed CommonEq ────── v0.2.1
   Installed CommonSolve ─── v0.2.4
   Installed PyCall ──────── v1.96.4
   Installed SymPy ───────── v2.3.3
   Installed SymPyCore ───── v0.3.1
    Updating `~/.julia/environments/v1.10/Project.toml`
  [24249f21] + SymPy v2.3.3
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [3709ef60] + CommonEq v0.2.1
  [38540f10] + CommonSolve v0.2.4
  [438e738f] + PyCall v1.96.4
  [24249f21] + SymPy v2.3.3
  [458b697b] + SymPyCore v0.3.1
  [8ea1fca8] + TermInterface v2.0.0
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`
Precompiling packages...
   3743.4 ms  ✓ CommonEq
   2136.5 ms  ✓ CommonSolve
   3070.5 ms  ✓ TermInterface
   7023.8 ms  ✓ SymPyCore
  25589.3 ms  ✓ PyCall
   2327.8 ms  ✓ SymPy
  6 dependencies successfully precompiled in

**Latexfy, SymPy, LinearAlgebra** を使う

In [2]:
using Latexify,SymPy,LinearAlgebra

$$A=\left(\begin{array}~ 1&1&0&0\\1&1&1&0\\1&1&1&1\\1&1&1&1\end{array}\right)$$として, $A$の累乗を求めたい。
ただ, 明らかに同じ成分の行があるので, 行列式は$0$になるし,
対角化は無理だ。

そこで, **Jordan標準形**を利用しようとなる。
他にも色々変形はあるのだろうけど。

そもそもが**Jordan標準形**とは対角成分だけでにできなくても,
対角に扱い易い正方行列が並ぶ次のような形様の行列。(縦横の線は「扱い易い正方行列」を見易くできるかと入れてありますが本来描くべきものではありません)
$$J=\left(\begin{array}{ccc|c|cc|c}~
\lambda_1&1&0&0&0&0&0\\
0&\lambda_1&1&0&0&0&0\\
0&0&\lambda_1&0&0&0&0\\\hline
0&0&0&\lambda_2&0&0&0\\\hline
0&0&0&0&\lambda_3&1&0\\
0&0&0&0&0&\lambda_3&0\\\hline
0&0&0&0&0&0&\lambda_4\\
\end{array}\right)$$
というものです。$\lambda_i$は固有値です。

この固有値は, 次のような行列の行列式が$0$になる値として設定されています。

そして任意の行列は必ずこのような**Jordan標準形**を持ち,
適切な行列$P$が存在して,
$$P^{-1}AP=J$$
を満たすと。これより,  
$$AP=PJ$$
であり, 更に
$$A=PJP^{-1}$$
が成り立っている。

そこで,
$$B=A-\lambda I=\left(\begin{array}{cccc}~ 1-\lambda&1&0&0\\1&1-\lambda&1&0\\1&1&1-\lambda&1\\1&1&1&1-\lambda\end{array}\right)$$
を用意しておきます。

In [3]:
A=SymPy.Matrix([1 1 0 0;1 1 1 0;1 1 1 1;1 1 1 1])
lambda=symbols("lambda",commutative = true)
B=A-lambda*1I(4)
B

4×4 Matrix{Sym{PyCall.PyObject}}:
 1 - λ           1           0           0
          1  1 - λ           1           0
          1           1  1 - λ           1
          1           1           1  1 - λ

この$B=A-\lambda I$の行列式を計算して貰い, ついでに因数分解もしてもらう。

In [4]:
factor(det(B))

 2                
λ ⋅(λ - 3)⋅(λ - 1)

$B=A-\lambda I$ の行列式が $0$ になるような $\lambda$ について解いて貰う... のだけど, 綺麗に因数分解できているので, 既に $\lambda=0,1,3$ であることは分かっている。

そして何より, 固有値が分かったので, **Jordan標準形**である$J$が
$$J=\left(\begin{array}{cc|c|c}~
\lambda_1&1&0&0\\
0&\lambda_1&0&0\\\hline
0&0&\lambda_2&0\\\hline
0&0&0&\lambda_3
\end{array}\right)=
\left(\begin{array}{cccc}~
0&1&0&0\\
0&0&0&0\\%\hline
0&0&1&0\\%\hline
0&0&0&3
\end{array}\right)$$
であることがこの時点で分かっていることとなる。

おそらくこの解を元にジョルダン標準形を拵えるのだったはず。

In [5]:
solve(det(B),lambda)

3-element Vector{Sym{PyCall.PyObject}}:
 0
 1
 3

**Claude** はこのあとそれぞれの解(固有値と呼んでいる)から**一般化固有ベクトル**を求めるとしている。

先ずは, $\lambda=0$に対して, ベクトル$v$で$Av=0$となるものを求めるようだ。

In [6]:
a,b,c,d,v=symbols("a,b,c,d,v",commutative = true)
v=[a;b;c;d]
zr=[0,0,0,0]
A*v

4-element Vector{Sym{PyCall.PyObject}}:
         a + b
     a + b + c
 a + b + c + d
 a + b + c + d

In [7]:
linsolve([a+b,a+b+c,a+b+c+d],(a,b,c))
v_1=[1,-1,0,0]

4-element Vector{Int64}:
  1
 -1
  0
  0

うーん... コレは**Julia**や**Python**だとどうすれば解けるのだろう。
上のセルに書いてある**linsolve([a+b,a+b+c,a+b+c+d],(a,b,c))**では解いてくれなかった。

解は明らかに$a=a,\ b=-a,\ c=0,\ d=0$ だ。

コレにより,
$$v_1=\left(\begin{array}{c}~1\\-1\\0\\0\end{array}\right)$$
$$Av_1=0$$
ということだ。

で, $0$が重解だからか？
次は, $Av_2=v_1$を解くのだけど...

In [8]:
A*v

4-element Vector{Sym{PyCall.PyObject}}:
         a + b
     a + b + c
 a + b + c + d
 a + b + c + d

仕方がないので, 見ながら解くと, $$v_1=[a,1-a,-2,1]$$となるか... コレにより,
$$v_2=\left(\begin{array}{c}~0\\1\\-2\\1\end{array}\right)$$
とするのか。で,
$Av_2=v_1$
となる。

次は, $\lambda=1$に対して, ベクトル$v$で $(A-1I)v=0$ となるものを求めるようだ。

In [9]:
(A-1I(4))*v

4-element Vector{Sym{PyCall.PyObject}}:
         b
     a + c
 a + b + d
 a + b + c

これも見ながら解けば, $$v_3=[a,0,-a,-a]$$となる。コレより,
$$v_3=\left(\begin{array}{c}~1\\0\\-1\\-1\end{array}\right)$$
とする。
$Av_3=v_3$となるわけだ。

次は, $\lambda=3$に対して, ベクトル$v$で $(A-3I)v=0$ となるものを求める。

In [10]:
(A-3I(4))*v

4-element Vector{Sym{PyCall.PyObject}}:
        -2⋅a + b
     a - 2⋅b + c
 a + b - 2⋅c + d
 a + b + c - 2⋅d

これも見ながら解けば, $$v_4=[a,2a,3a,3a]$$となる。コレより,
$$v_4=\left(\begin{array}~1\\2\\3\\3\end{array}\right)$$
とする。
$Av_4=3v_4$となるわけだ。

あとは, ここまでで得た, $v_1,v_2,v_3,v_4$を並べれば次のように$P$が構成できるのだと。
$$P=\left(\begin{array}{cccc}~1&0&1&1\\-1&1&0&2\\0&-2&-1&3\\0&1&-1&3\end{array}\right)$$
どういう具合だろう？

つまり, $P^{-1}AP=J$ つまり $AP=PJ$ なのだが,  
\begin{align*}
Av_1&=0\\
Av_2&=v_1\\
Av_3&=Iv_3=v_3\\
Av_4&=3Iv_4=3v_4\\
\end{align*}
だから,
\begin{align*}
AP=\left(\begin{array}~0&v_1&v_2&3v_3\end{array}\right)
\end{align*}


In [11]:
P=SymPy.Matrix([sympy.sqrt(1) 0 1 1;-1 1 0 2;0 -2 -1 3;0 1 -1 3])

4×4 Matrix{Sym{PyCall.PyObject}}:
  1   0   1  1
 -1   1   0  2
  0  -2  -1  3
  0   1  -1  3

$P$の逆行列は求められるので, あとは$P^{-1}AP$を計算すれば**Jordan標準形**$J$が得られる。

In [12]:
inv(P)

4×4 Matrix{Sym{PyCall.PyObject}}:
 1/3  -2/3  -1/9   4/9
   0     0  -1/3   1/3
 1/2   1/2     0  -1/2
 1/6   1/6   1/9  1/18

In [13]:
A*P

4×4 Matrix{Sym{PyCall.PyObject}}:
 0   1   1  3
 0  -1   0  6
 0   0  -1  9
 0   0  -1  9

固有値$0$が重解なので, $0$が多い行列になっている。スカスカだ。

In [14]:
J=inv(P)*A*P

4×4 Matrix{Sym{PyCall.PyObject}}:
 0  1  0  0
 0  0  0  0
 0  0  1  0
 0  0  0  3

そして当然だが, このスカスカの行列$J$に$P$と$P^{-1}$を掛ければ$A$となる。

In [15]:
P*J*inv(P)

4×4 Matrix{Sym{PyCall.PyObject}}:
 1  1  0  0
 1  1  1  0
 1  1  1  1
 1  1  1  1

In [16]:
J^2

4×4 Matrix{Sym{PyCall.PyObject}}:
 0  0  0  0
 0  0  0  0
 0  0  1  0
 0  0  0  9

In [17]:
J^3

4×4 Matrix{Sym{PyCall.PyObject}}:
 0  0  0   0
 0  0  0   0
 0  0  1   0
 0  0  0  27

In [18]:
J^4

4×4 Matrix{Sym{PyCall.PyObject}}:
 0  0  0   0
 0  0  0   0
 0  0  1   0
 0  0  0  81

明らかに, $J^n=\begin{pmatrix}0&0&0&0\\0&0&0&0\\0&0&1&0\\0&0&0&3^n\\\end{pmatrix}$だと分かる。すると, $A^n=PJ^nP^{-1}$だから,

In [28]:
n=symbols("n",commutative = true)
JJ=[0 0 0 0;0 0 0 0;0 0 1 0;0 0 0 3^n]

4×4 Matrix{Sym{PyCall.PyObject}}:
 0  0  0    0
 0  0  0    0
 0  0  1    0
 0  0  0  3^n

In [37]:
AA=P*JJ*inv(P)
simplify.(AA)

4×4 Matrix{Sym{PyCall.PyObject}}:
 3^n/6 + 1/2  3^n/6 + 1/2    3^(n - 2)  3^n/18 - 1/2
   3^(n - 1)    3^(n - 1)  2*3^(n - 2)     3^(n - 2)
 3^n/2 - 1/2  3^n/2 - 1/2    3^(n - 1)   3^n/6 + 1/2
 3^n/2 - 1/2  3^n/2 - 1/2    3^(n - 1)   3^n/6 + 1/2

In [44]:
subs.(AA,(n=>10))

4×4 Matrix{Sym{PyCall.PyObject}}:
  9842   9842   6561  3280
 19683  19683  13122  6561
 29524  29524  19683  9842
 29524  29524  19683  9842